In [1]:
import time
import datetime
from datetime import datetime

import pickle

import pyspark
import findspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, pandas_udf, PandasUDFType, lit, avg
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession, DataFrame
import pyspark.pandas as ps

from delta import *
from delta.tables import *

import pyarrow as pa

from collections import defaultdict

import itertools
import traceback
import re
import psutil
import requests
import string
import uuid

from more_itertools import unique_everseen

import pandas as pd
import numpy as np
import json
from numpyencoder import NumpyEncoder

from tqdm import tqdm
from unidecode import unidecode

/home/hd_micula/.local/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
print("LOG: BUILDING SESSION...")

LOG: BUILDING SESSION...


In [3]:
builder = (pyspark.sql.SparkSession.builder.appName("JsonsGenerator")\
.config('spark.driver.memory',"64G")\
.config('spark.master',"spark://hadoop:7077")\
.config("spark.executor.cores", "16")\
.config("spark.executor.memory", "70G")\
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
.config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog"))
spark = configure_spark_with_delta_pip(builder).getOrCreate()

24/05/08 14:43:53 WARN SparkConf: The configuration key 'spark.yarn.executor.memoryOverhead' has been deprecated as of Spark 2.3 and may be removed in the future. Please use the new key 'spark.executor.memoryOverhead' instead.


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml

In [5]:
# JSON METADATA
# Get current date for versioning
now = datetime.now()
date_time = now.strftime("%m %d %Y %H %M %S")
date_time_string = str(datetime.now())
creation_timestamp = date_time
json_version = '1.0.16'
general_source_type = "OPI" # or "AUTHOR"

In [ ]:
# Zadanie polega na zlepieniu różnych autorów tej samej publikacji do tego samego obiektu, wcześniej sobie te dane zagregowałem i wrzuciłem w słownik,
with open('uuid_2_persons.pickle', 'rb') as handle:
    uuid_2_persons = pickle.load(handle)

In [28]:
def get_max(a, b, c):
    try:
        a = int(a)
    except (TypeError, ValueError) as e:
        a = 0
    try:
        b = int(b)
    except (TypeError, ValueError) as e:
        b = 0
    try:
        c = int(c)
    except (TypeError, ValueError) as e:
        c = 0

    return max(a, b, c)

In [29]:
acceptable_chars = set(string.digits + '-' + '=' + ',')

def pages_acceptable_chars(str):
   validation = set(str)
   if validation.issubset(acceptable_chars):
      return True
   else:
      return False

def split_pages(pages):
    pages_from = pages
    pages_to = None

    if pages is None:
        return pages_from, pages_to
    
    if pages_acceptable_chars(pages):
        if len(re.split(', |_|-|!', pages)) == 2:
            pages_from, pages_to = re.split(', |_|-|!', pages)
        
    return pages_from, pages_to

In [30]:
def stringify_json(_json):
    json_string ='{"in_json": ' + str(json.dumps(_json, cls=NumpyEncoder, ensure_ascii=False)) + '}'
    json_string = json_string.encode('utf-8')
    
    return json_string

In [31]:
def check_if_all_nulls(d):
    return all(value == None for value in d.values())

def remove_nested_none_objects(d):
    for k, v in list(d.items()):
        if isinstance(v, list):
            new_v = []
            for e in v:
                if isinstance(e, dict):
                    remove_nested_none_objects(e)
                if isinstance(e, dict) and not check_if_all_nulls(e):
                    new_v.append(e)
            d[k] = new_v
        
        if isinstance(v, dict):
            if all(value == None for value in v.values()):
                d[k] = None
                remove_nested_none_objects(d)
            else:
                remove_nested_none_objects(v)
        if v is None:
            d[k] = None
    return d

In [32]:
def change_dateformat(date, in_format='%y/%m/%d', out_format='%Y-%m-%d'):
    return datetime.strptime(date, in_format).strftime(out_format)

In [34]:
# Chciałem proces napisany w pandasie przerobić na Sparka rozsyłając DataFramy pandasa na workerów, docelowo całkowicie przepnę się na Sparka
data = pd.read_csv('data.csv')

/tmp/ipykernel_2989769/3931131315.py:2: DtypeWarning: Columns (0,1,5,36,44,51,55,56,58,80,88,111,112,115,116,117,120,121,122,123,128,129,132,135,137,138,139,146,150) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data.csv')


In [37]:
import pandas as pd
import pyarrow as pa
hdfs_config = {
     "host" : "",
     "port" : 9000,
     "user" : "user"
}
fs = pa.hdfs.connect(hdfs_config['host'], hdfs_config['port'], 
user=hdfs_config['user'])

/tmp/ipykernel_2989769/3147596676.py:8: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  fs = pa.hdfs.connect(hdfs_config['host'], hdfs_config['port'],


In [38]:
data_abstracts = pd.read_csv(fs.open("/hd_micula/pbn_dump/abstracts.csv"))
data_keywords = pd.read_csv(fs.open("/hd_micula/pbn_dump/keywords.csv"))
data_other_titles = pd.read_csv(fs.open("/hd_micula/pbn_dump/other_titles.csv"))

data_abstracts = data_abstracts.replace(np.nan, None)
data_keywords = data_keywords.replace(np.nan, None)
data_other_titles = data_other_titles.replace(np.nan, None)

In [41]:
with open('new_mapping_hsks_04 29 2024 18 46 52.pickle', 'rb') as handle:
    new_mapping_hsks = pickle.load(handle)

with open('mapping_object_id_to_uuid_04 29 2024 18 46 52.pickle', 'rb') as handle:
    mapping_object_id_to_uuid = pickle.load(handle)

In [46]:
def gen_json_wrap(df, data=data, action_type="INSERT"):
    @pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)
    def gen_json(pdf):
        uuid_name = pdf['NEW_UUID'].values[0]
        pub_group = pdf
        action_type = "INSERT"

        best_id = None
        group = None
        _json = {}
        excep = ""
        global_id = ""
        
        try:
            # Get best ID and group
            # group = pub_group['OBJECT_ID_PUBLICATION'].tolist()
            group = pub_group['OBJECT_ID'].tolist()
            if group[0] in new_mapping_hsks.keys():
                best_id = new_mapping_hsks[group[0]]
            else:
                best_id = group[0]
            if group[0] in mapping_object_id_to_uuid.keys():
                global_id = mapping_object_id_to_uuid[group[0]]
            else:
                global_id = group[0]
            deduplication_publication_group_identifier = uuid_name
    
            author_profile_idn = None
        
            # Get json metadata
            now = datetime.now()
            date_time = now.strftime("%m %d %Y %H %M %S")
            date_time_string = str(now)
            creation_timestamp = date_time
           
            # To jest na pewno do zmiany.
            # Filter DF for this publication
            df_filtered_pd = data[data['OBJECT_ID_PUBLICATION'].isin(group)]
        
            # PERSONS
            persons_data = []
        
            # Iterate over persons
            for person in uuid_2_persons[uuid_name]:
                first_name = person['first_name']
                last_name = person['last_name']
                cid = person['cid']
                idn = person['idn']
                affiliation_institution_name = person['affiliation_institution_name']
                affiliation_uuid_publication = person['affiliation_uuid_publication']
                contribution_order = person['contribution_order']
                person_role = person['person_role']
                person_hsk = person['person_hsk']
                new_person_hsk = person['new_person_hsk']

                try:
                    first_name_split = person['first_name'].split(' ', 1)
                    if len(first_name_split) == 1:
                        first_name = first_name_split[0]
                        middle_name = None
                    else:
                        first_name = first_name_split[0]
                        middle_name = first_name_split[1]
                except:
                    pass

                try:
                    last_name_split = person['last_name'].split(' ', 1)
                except:
                    pass
    
                second_name = person['last_name']
                last_name_prefix = None
            
                organization_unit_name = person['affiliation_institution_name']
                organization_unit_uuid = person['affiliation_uuid_publication']
                statement_date = None
                
                cf_pers_sequence = person['contribution_order']
                try:
                    if cf_pers_sequence is not null and float('nan') :
                        cf_pers_sequence = int(cf_pers_sequence)
                except:
                    pass
                    
                cf_pers_identif = []
            
                _publication_role = person['person_role']
                publication_role_id = dict_publication_role[_publication_role]['id']
                publication_role_code = dict_publication_role[_publication_role]['code']
                publication_role_value_pol = dict_publication_role[_publication_role]['label_pl']
                publication_role_value_eng = dict_publication_role[_publication_role]['label_en']
                
                deduplication_person_group_identifier = None
                
                new_person_hsk = person['new_person_hsk']
                person_identifiers = person['identifiers']
        
                for person_identifier in person_identifiers:
                    _source = 'PBN'
                    identifier = person_identifier
                    dic_golden_record_source_id = dict_golden_record_source[_source]['id']
                    dic_golden_record_source_code = dict_golden_record_source[_source]['code']
                    dic_golden_record_source_value_pol = dict_golden_record_source[_source]['label_pl']
                    dic_golden_record_source_value_eng = dict_golden_record_source[_source]['label_en']
                    
                    x = {
                        "identifier": identifier,
                        "dicGoldenRecordSource": {
                            "dicGoldenRecordSourceId": dic_golden_record_source_id,
                            "dicGoldenRecordSourceCode": dic_golden_record_source_code,
                            "dicGoldenRecordSourceLabelPl": dic_golden_record_source_value_pol,
                            "dicGoldenRecordSourceLabelEn": dic_golden_record_source_value_eng
                        }
                    }
                    cf_pers_identif.append(x)
    
                cf_pers_identif=list(unique_everseen(cf_pers_identif))
    
                organizations = []
                organization_json = {
                    "cfOrgUnitName": {
                        "organizationUnitName": organization_unit_name,
                        "organizationUnitUuid": organization_unit_uuid   
                    },
                    "cfOrgUnitResPublStatement": {
                        "organizationUnitName": organization_unit_name,
                        "organizationUnitUuid": organization_unit_uuid,
                        "statementDate": statement_date
                    },
                }
                organizations.append(organization_json)
            
                persons_data.append(
                    {
                            "cfPers": {
                                "cid": cid,
                                "firstName": first_name,
                                "middleName": middle_name,
                                "lastName": second_name,
                                "lastNamePrefix": last_name_prefix,
                                "idn": idn
                            },
                            "organizationsData": organizations,
                            "cfPersPublication": {
                                "cfPersSequence": cf_pers_sequence
                            },
                            "cfPersPublicationRole": {
                                "publicationRoleId": publication_role_id,
                                "publicationRoleCode": publication_role_code,
                                "publicationRoleLabelPl": publication_role_value_pol,
                                "publicationRoleLabelEn": publication_role_value_eng
                            },
                            "cfPersIdentif": cf_pers_identif,
                            "persGroupId": deduplication_person_group_identifier
                        },
                )
        
            # PUBLICATION
            row = df_filtered_pd[df_filtered_pd['OBJECT_ID_PUBLICATION'] == best_id].iloc[0]
        
            # Publication metadata
            page_number = row['PAGES_METADATA']
            year = row['YEAR_PUBLICATION']
            if year:
                year = int(year)
            page_from_to = row['PAGES_FROM_TO_METADATA']
            page_from = page_from_to
            page_to = None
            try:
                page_from, page_to = split_pages(page_from_to)
            except:
                pass
            doi = row['DOI_METADATA']
            ismn = row['ISMN_METADATA']
            translation = row['TRANSLATION_METADATA']
            if translation is not None:
                translation = bool(translation)
            url = row['PUBLIC_URI_METADATA']
            volume_number = row['VOLUME_METADATA']
            issue_number = row['ISSUE_METADATA']
            publishing_series = row['SERIES_METADATA']
            publishing_series_number = row['NUMBER_IN_SERIES_METADATA']
            publishing_number = row['EDITION_NUMBER_METADATA']
            publication_place = row['PUBLICATION_PLACE_METADATA']
            publication_main_language = row['MAIN_LANGUAGE_METADATA']
            publication_original_language = row['ORIGINAL_LANGUAGE_METADATA']
            original_language = row['ORIGINAL_LANGUAGE_METADATA']
            _lang_name = row["MAIN_LANGUAGE_METADATA"] # or row["ORIGINAL_LANGUAGE_METADATA"]
            publication_cf_lang_name_id = dict_cf_lang_name_pl[_lang_name]['id']
            publication_cf_lang_name_code = dict_cf_lang_name_pl[_lang_name]['code']
            publication_cf_lang_name_value_pol = dict_cf_lang_name_pl[_lang_name]['language_pl']
            publication_cf_lang_name_value_eng = dict_cf_lang_name_pl[_lang_name]['language_en']
            publication_cf_lang_name_value_ger = dict_cf_lang_name_pl[_lang_name]['language_de']
            publication_cf_lang_name_value_fre = dict_cf_lang_name_pl[_lang_name]['language_fr']
            publication_cf_lang_name_iso_639_2 = dict_cf_lang_name_pl[_lang_name]['iso_639_2']
            publication_cf_lang_name_iso_639_1 = dict_cf_lang_name_pl[_lang_name]['iso_639_1']
            publication_cf_lang_name_wikiurl = dict_cf_lang_name_pl[_lang_name]['wikiurl']
            _publication_type = row['PUBLICATION_TYPE_PUBLICATION']
            dic_publication_type_id = dict_publication_type[_publication_type]['id']
            dic_publication_type_code = dict_publication_type[_publication_type]['code']
            dic_publication_type_value_pol = dict_publication_type[_publication_type]['label_pl']
            dic_publication_type_value_eng = dict_publication_type[_publication_type]['label_en']
            issn = row['ISSN_METADATA']
            _source_type = general_source_type
            dic_source_type_id = dict_source_type[_source_type]['id']
            dic_source_type_code = dict_source_type[_source_type]['code']
            dic_source_type_value_pol = dict_source_type[_source_type]['label_pl']
            dic_source_type_value_eng = dict_source_type[_source_type]['label_en']
        
            # ABSTRACTS
            # To tez jest do zmiany
            abstracts_df = data_abstracts[data_abstracts['OBJECT_ID'].isin(group)].drop_duplicates(subset=['TYPE', 'ORDINAL_NUMBER_ABSTRACT', 'LANGUAGE_CODE', 'CONTENT'])
            cf_res_publ_abstract = []
            
            for _, row in abstracts_df.iterrows():
                abstract_content = row['CONTENT']
                _abstract_lang_name = row['LANGUAGE_CODE']
                abstract_cf_lang_name_id = dict_cf_lang_name[_abstract_lang_name]['id']
                abstract_cf_lang_name_code = dict_cf_lang_name[_abstract_lang_name]['code']
                abstract_cf_lang_name_value_pol = dict_cf_lang_name[_abstract_lang_name]['language_pl']
                abstract_cf_lang_name_value_eng = dict_cf_lang_name[_abstract_lang_name]['language_en']
                abstract_cf_lang_name_value_ger = dict_cf_lang_name[_abstract_lang_name]['language_de']
                abstract_cf_lang_name_value_fre = dict_cf_lang_name[_abstract_lang_name]['language_fr']
                abstract_cf_lang_name_iso_639_2 = dict_cf_lang_name[_abstract_lang_name]['iso_639_2']
                abstract_cf_lang_name_iso_639_1 = dict_cf_lang_name[_abstract_lang_name]['iso_639_1']
                abstract_cf_lang_name_wikiurl = dict_cf_lang_name[_abstract_lang_name]['wikiurl']
                
                _abstract_publication_type = row['TYPE']
                abstract_dic_source_type_id = dict_publication_type[_abstract_publication_type]['id']
                abstract_dic_source_type_code = dict_publication_type[_abstract_publication_type]['code']
                abstract_dic_source_type_value_pol = dict_publication_type[_abstract_publication_type]['label_pl']
                abstract_dic_source_type_value_eng = dict_publication_type[_abstract_publication_type]['label_en']

                _abstract_source_type = general_source_type
                abstract_dic_source_type_id = dict_source_type[_abstract_source_type]['id']
                abstract_dic_source_type_code = dict_source_type[_abstract_source_type]['code']
                abstract_dic_source_type_value_pol = dict_source_type[_abstract_source_type]['label_pl']
                abstract_dic_source_type_value_eng = dict_source_type[_abstract_source_type]['label_en']
            
                if not translation:
                    if _abstract_lang_name == dict_cf_lang_name_pl[publication_main_language]['code']:
                        abstract_translation = 'O'
                    else:
                        abstract_translation = 'H'
                if translation:
                    if _abstract_lang_name == dict_cf_lang_name_pl[publication_original_language]['code']:
                        abstract_translation = 'O'
                    else:
                        abstract_translation = 'H'
                        
                
                cf_res_publ_abstract.append(
                    {
                            "content": abstract_content,
                            "cfLangName": {
                                "cfLangNameId": abstract_cf_lang_name_id,
                                "cfLangNameCode": abstract_cf_lang_name_code,
                                "cfLangNameLanguagePl": abstract_cf_lang_name_value_pol,
                                "cfLangNameLanguageEn": abstract_cf_lang_name_value_eng,
                                "cfLangNameLanguageDe": abstract_cf_lang_name_value_ger,
                                "cfLangNameLanguageFr": abstract_cf_lang_name_value_fre,
                                "cfLangNameIso639_2": abstract_cf_lang_name_iso_639_2,
                                "cfLangNameIso639_1": abstract_cf_lang_name_iso_639_1,
                                "cfLangNameWikiurl": abstract_cf_lang_name_wikiurl
                            },
                            "dicSourceType": {
                                "dicSourceTypeId": abstract_dic_source_type_id,
                                "dicSourceTypeCode": abstract_dic_source_type_code,
                                "dicSourceTypeLabelPl": abstract_dic_source_type_value_pol,
                                "dicSourceTypeLabelEn": abstract_dic_source_type_value_eng
                            },
                            "translation": abstract_translation
                        }
                )
            # BIBL NOTE
            cf_res_publ_bibl_note = []
            bibliographical_notes = []
            
            bibliography_record = None
            bibl_translation = None
            
            for bibliographical_note in bibliographical_notes:
                note = {
                    "bibliographyRecord": bibliography_record,
                    "translation": bibl_translation
                }
                cf_res_publ_bibl_note.append(note)
        
            # KEYWORDS
            # To tez jest do zmiany
            keywords_df = data_keywords[data_keywords['OBJECT_ID'].isin(group)].drop_duplicates(subset=['PUBLICATION_TYPE', 'ORDINAL_NUMBER_LANGUAGE', 'LANGUAGE_CODE', 'ORDINAL_NUMBER_KEYWORDS', 'KEYWORDS'])
            
            cf_res_publ_keyw = []
            
            for _, row in keywords_df.iterrows():
                keywords = row['KEYWORDS']
                ordinal_numer = row['ORDINAL_NUMBER_KEYWORDS']
            
                _keywords_lang_name = row['LANGUAGE_CODE']
                keywords_cf_lang_name_id = dict_cf_lang_name[_keywords_lang_name]['id']
                keywords_cf_lang_name_code = dict_cf_lang_name[_keywords_lang_name]['code']
                keywords_cf_lang_name_value_pol = dict_cf_lang_name[_keywords_lang_name]['language_pl']
                keywords_cf_lang_name_value_eng = dict_cf_lang_name[_keywords_lang_name]['language_en']
                keywords_cf_lang_name_value_ger = dict_cf_lang_name[_keywords_lang_name]['language_de']
                keywords_cf_lang_name_value_fre = dict_cf_lang_name[_keywords_lang_name]['language_fr']
                keywords_cf_lang_name_iso_639_2 = dict_cf_lang_name[_keywords_lang_name]['iso_639_2']
                keywords_cf_lang_name_iso_639_1 = dict_cf_lang_name[_keywords_lang_name]['iso_639_1']
                keywords_cf_lang_name_wikiurl = dict_cf_lang_name[_keywords_lang_name]['wikiurl']
            
                _source_type = general_source_type
                dic_source_type_id = dict_source_type[_source_type]['id']
                dic_source_type_code = dict_source_type[_source_type]['code']
                dic_source_type_value_pol = dict_source_type[_source_type]['label_pl']
                dic_source_type_value_eng = dict_source_type[_source_type]['label_en']
            
                if not translation:
                    if _keywords_lang_name == dict_cf_lang_name_pl[publication_main_language]['code']:
                        keywords_translation = 'O'
                    else:
                        keywords_translation = 'H'
                if translation:
                    if _keywords_lang_name == dict_cf_lang_name_pl[publication_original_language]['code']:
                        keywords_translation = 'O'
                    else:
                        keywords_translation = 'H'
                
                keywords_json = {
                    "keyWords": keywords,
                    "ordinalNumber": ordinal_numer,
                    "cfLangName": {
                        "cfLangNameId": keywords_cf_lang_name_id,
                        "cfLangNameCode": keywords_cf_lang_name_code,
                        "cfLangNameLanguagePl": keywords_cf_lang_name_value_pol,
                        "cfLangNameLanguageEn": keywords_cf_lang_name_value_eng,
                        "cfLangNameLanguageDe": keywords_cf_lang_name_value_ger,
                        "cfLangNameLanguageFr": keywords_cf_lang_name_value_fre,
                        "cfLangNameIso639_2": keywords_cf_lang_name_iso_639_2,
                        "cfLangNameIso639_1": keywords_cf_lang_name_iso_639_1,
                        "cfLangNameWikiurl": keywords_cf_lang_name_wikiurl
                    },
                    "dicSourceType": {
                        "dicSourceTypeId": dic_source_type_id,
                        "dicSourceTypeCode": dic_source_type_code,
                        "dicSourceTypeLabelPl": dic_source_type_value_pol,
                        "dicSourceTypeLabelEn": dic_source_type_value_eng
                    },
                    "translation": keywords_translation
                }
                
                cf_res_publ_keyw.append(keywords_json)
        
            # OTHER TITLES
            # To tez jest do zmiany, jak wyzej
            other_titles_df = data_other_titles[data_other_titles['OBJECT_ID'].isin(group)].drop_duplicates(subset=['PUBLICATION_TYPE', 'ORDINAL_NUMBER_OTHER_TITLE', 'LANGUAGE_CODE', 'TITLE'])
            cf_res_title = []
            
            # Main title
            row = df_filtered_pd[df_filtered_pd['OBJECT_ID_PUBLICATION'] == best_id].iloc[0]
            title = row['TITLE_METADATA']
            main_title = True
            
            _lang_name = row["MAIN_LANGUAGE_METADATA"] # or row["ORIGINAL_LANGUAGE_METADATA"]
            cf_lang_name_id = dict_cf_lang_name_pl[_lang_name]['id']
            cf_lang_name_code = dict_cf_lang_name_pl[_lang_name]['code']
            cf_lang_name_value_pol = dict_cf_lang_name_pl[_lang_name]['language_pl']
            cf_lang_name_value_eng = dict_cf_lang_name_pl[_lang_name]['language_en']
            cf_lang_name_value_ger = dict_cf_lang_name_pl[_lang_name]['language_de']
            cf_lang_name_value_fre = dict_cf_lang_name_pl[_lang_name]['language_fr']
            cf_lang_name_iso_639_2 = dict_cf_lang_name_pl[_lang_name]['iso_639_2']
            cf_lang_name_iso_639_1 = dict_cf_lang_name_pl[_lang_name]['iso_639_1']
            cf_lang_name_wikiurl = dict_cf_lang_name_pl[_lang_name]['wikiurl']
            
            _source_type = general_source_type
            dic_source_type_id = dict_source_type[_source_type]['id']
            dic_source_type_code = dict_source_type[_source_type]['code']
            dic_source_type_value_pol = dict_source_type[_source_type]['label_pl']
            dic_source_type_value_eng = dict_source_type[_source_type]['label_en']
        
            if not translation:
                if dict_cf_lang_name_pl[_lang_name]['code'] == dict_cf_lang_name_pl[publication_main_language]['code']:
                    main_title_translation = 'O'
                else:
                    main_title_translation = 'H'
            if translation:
                if dict_cf_lang_name_pl[_lang_name]['code'] == dict_cf_lang_name_pl[publication_original_language]['code']:
                    main_title_translation = 'O'
                else:
                    main_title_translation = 'H'
            
            title_json = {
                "title": title,
                "mainTitle": main_title,
                "cfLangName": {
                    "cfLangNameId": cf_lang_name_id,
                    "cfLangNameCode": cf_lang_name_code,
                    "cfLangNameLanguagePl": cf_lang_name_value_pol,
                    "cfLangNameLanguageEn": cf_lang_name_value_eng,
                    "cfLangNameLanguageDe": cf_lang_name_value_ger,
                    "cfLangNameLanguageFr": cf_lang_name_value_fre,
                    "cfLangNameIso639_2": cf_lang_name_iso_639_2,
                    "cfLangNameIso639_1": cf_lang_name_iso_639_1,
                    "cfLangNameWikiurl": cf_lang_name_wikiurl
                },
                "dicSourceType": {
                    "dicSourceTypeId": dic_source_type_id,
                    "dicSourceTypeCode": dic_source_type_code,
                    "dicSourceTypeLabelPl": dic_source_type_value_pol,
                    "dicSourceTypeLabelEn": dic_source_type_value_eng
                },
                "translation": main_title_translation
            }
            cf_res_title.append(title_json)
            
            # Other titles
            for _, row in other_titles_df.iterrows():
                title = row['TITLE']
                main_title = False
                
                _lang_name = row["LANGUAGE_CODE"] # or row["ORIGINAL_LANGUAGE_METADATA"]
                cf_lang_name_id = dict_cf_lang_name[_lang_name]['id']
                cf_lang_name_code = dict_cf_lang_name[_lang_name]['code']
                cf_lang_name_value_pol = dict_cf_lang_name[_lang_name]['language_pl']
                cf_lang_name_value_eng = dict_cf_lang_name[_lang_name]['language_en']
                cf_lang_name_value_ger = dict_cf_lang_name[_lang_name]['language_de']
                cf_lang_name_value_fre = dict_cf_lang_name[_lang_name]['language_fr']
                cf_lang_name_iso_639_2 = dict_cf_lang_name[_lang_name]['iso_639_2']
                cf_lang_name_iso_639_1 = dict_cf_lang_name[_lang_name]['iso_639_1']
                cf_lang_name_wikiurl = dict_cf_lang_name[_lang_name]['wikiurl']
                
                _source_type = general_source_type
                dic_source_type_id = dict_source_type[_source_type]['id']
                dic_source_type_code = dict_source_type[_source_type]['code']
                dic_source_type_value_pol = dict_source_type[_source_type]['label_pl']
                dic_source_type_value_eng = dict_source_type[_source_type]['label_en']
                
                if not translation:
                    if _lang_name == dict_cf_lang_name_pl[publication_main_language]['code']:
                        other_title_translation = 'O'
                    else:
                        other_title_translation = 'H'
                if translation:
                    if _lang_name == dict_cf_lang_name_pl[publication_original_language]['code']:
                        other_title_translation = 'O'
                    else:
                        other_title_translation = 'H'
                
                title_json = {
                    "title": title,
                    "mainTitle": main_title,
                    "cfLangName": {
                        "cfLangNameId": cf_lang_name_id,
                        "cfLangNameCode": cf_lang_name_code,
                        "cfLangNameLanguagePl": cf_lang_name_value_pol,
                        "cfLangNameLanguageEn": cf_lang_name_value_eng,
                        "cfLangNameLanguageDe": cf_lang_name_value_ger,
                        "cfLangNameLanguageFr": cf_lang_name_value_fre,
                        "cfLangNameIso639_2": cf_lang_name_iso_639_2,
                        "cfLangNameIso639_1": cf_lang_name_iso_639_1,
                        "cfLangNameWikiurl": cf_lang_name_wikiurl
                    },
                    "dicSourceType": {
                        "dicSourceTypeId": dic_source_type_id,
                        "dicSourceTypeCode": dic_source_type_code,
                        "dicSourceTypeLabelPl": dic_source_type_value_pol,
                        "dicSourceTypeLabelEn": dic_source_type_value_eng
                    },
                    "translation": other_title_translation
                }
                cf_res_title.append(title_json)
            
            row = df_filtered_pd[df_filtered_pd['OBJECT_ID_PUBLICATION'] == best_id].iloc[0]
            
            journal_name = row['TITLE_JOURNAL']
            journal_issn = row['ISSN_JOURNAL']
            journal_eissn = row['EISSN_JOURNAL']
            journal_points = row['POINTS_JOURNAL_POINTS']
            journalIsPoints = row['MNISW_ID_JOURNAL']
            if journalIsPoints is not None:
                journalIsPoints = bool(journalIsPoints)
            if journalIsPoints is None:
                journalIsPoints = False
            journal_url = row['WEBSITE_LINK_JOURNAL']
            journal_publisher_name = None
            journal_publisher_points = None
            if journal_publisher_points is not None:
                journal_publisher_points = bool(journal_publisher_points)
            if journal_publisher_points is None:
                journal_publisher_points = False
            journal_publisher_status = None
    
            if journal_publisher_status is None:
                journal_publisher_status = 'ACTIVE'
                
            
            conference_series_name = row['FULL_NAME_CONFERENCE_SERIES']
            conference_series_points = row['POINTS_CONFERENCE_POINTS']
            conferenceSeriesIsPoints = row['MNISW_ID_CONFERENCE_SERIES']
            if conferenceSeriesIsPoints is not None:
                conferenceSeriesIsPoints = bool(conferenceSeriesIsPoints)
            if conferenceSeriesIsPoints is None:
                conferenceSeriesIsPoints = False
            conference_series_short_name = row['SHORT_NAME_CONFERENCE_SERIES']
            conference_series_url = row['WEBSITE_CONFERENCE_SERIES']
            
            conference_event_full_name = row['FULL_NAME_CONFERENCE_EDITION']
            conference_event_start_date = row['START_DATE_CONFERENCE_EDITION']
            try:
                conference_event_start_date = change_dateformat(conference_event_start_date)
            except:
                pass
            conference_event_place = row['CITY_CONFERENCE_EDITION']
            conference_event_end_date = row['END_DATE_CONFERENCE_EDITION']
            try:
                conference_event_end_date = change_dateformat(conference_event_end_date)
            except:
                pass
            conference_event_url = row['WEBSITE_CONFERENCE_EDITION']
            conference_event_short_name = row['SHORT_NAME_CONFERENCE_EDITION']
            
            _conference_country = row['COUNTRY_CONFERENCE_EDITION']
            conference_event_country_id = dict_country_pl[_conference_country]['id']
            conference_event_country_code = dict_country_pl[_conference_country]['code']
            conference_event_country_value_pol = dict_country_pl[_conference_country]['label_pl']
            conference_event_country_value_eng = dict_country_pl[_conference_country]['label_en']
            
            _conference_event_type = None
            conference_event_conference_event_type_id = None
            conference_event_conference_event_type_code = None
            conference_event_conference_event_type_value_pol = None
            conference_event_conference_event_type_value_eng = None
            
            publisher_name = row['NAME_PUBLISHER']
            publisher_points = row['POINTS_PUBLISHER_POINTS']
            publisherIsPoints = row['MNISW_ID_PUBLISHER']
            if publisherIsPoints is not None:
                publisherIsPoints = bool(publisherIsPoints)
            if publisherIsPoints is None:
                publisherIsPoints = False
            publisher_status = None
            if publisher_status is None:
                publisher_status = 'ACTIVE'
            
            publisher_isbn = row['ISBN_METADATA']
    
            _publishing_format = None
            publisher_publishing_format_id = None
            publisher_publishing_format_code = None
            publisher_publishing_format_value_pol = None
            publisher_publishing_format_value_eng = None
            
            publisher_reedition = None
            publisher_publishing_leadership = None
            publishing_platform_name = None
    
            publishers_data = []
    
            publishers_json = {
                "publisher": {
                    "name": publisher_name,
                    "points": publisherIsPoints,
                    "status": publisher_status
                },
                "resPubPublisher": {
                    "isbn": publisher_isbn,
                    "publishingFormat": {
                        "publishingFormatId": publisher_publishing_format_id,
                        "publishingFormatCode": publisher_publishing_format_code,
                        "publishingFormatLabelPl": publisher_publishing_format_value_pol,
                        "publishingFormatLabelEn": publisher_publishing_format_value_eng
                    },
                    "reedition": publisher_reedition,
                    "publishingLeadership": publisher_publishing_leadership
                }
            }
    
            publishers_data.append(publishers_json)
        
            points_points = get_max(row['POINTS_CONFERENCE_POINTS'], row['POINTS_JOURNAL_POINTS'], row['POINTS_PUBLISHER_POINTS'])
            points_year = get_max(row['YEAR_POINTS_CONFERENCE_POINTS'], row['YEAR_POINTS_JOURNAL_POINTS'], row['YEAR_POINTS_PUBLISHER_POINTS'])
            
            _res_pub_open_access_license = row['OPEN_ACCESS_LICENSE_METADATA']
            res_pub_open_access_dic_license_id = dict_license[_res_pub_open_access_license]['id']
            res_pub_open_access_dic_license_code = dict_license[_res_pub_open_access_license]['code']
            res_pub_open_access_dic_license_pol = dict_license[_res_pub_open_access_license]['label_pl']
            res_pub_open_access_dic_license_eng = dict_license[_res_pub_open_access_license]['label_en']
            
            _res_pub_open_access_text_version = row['OPEN_ACCESS_TEXT_VERSION_METADATA']
            res_pub_open_access_dic_text_version_id = dict_text_version[_res_pub_open_access_text_version]['id']
            res_pub_open_access_dic_text_version_code = dict_text_version[_res_pub_open_access_text_version]['code']
            res_pub_open_access_dic_text_version_pol = dict_text_version[_res_pub_open_access_text_version]['label_pl']
            res_pub_open_access_dic_text_version_eng = dict_text_version[_res_pub_open_access_text_version]['label_en']
            
            _res_pub_open_access_share_method = row['OPEN_ACCESS_MODE_METADATA']
            res_pub_open_access_dic_share_method_id = dict_share_method[_res_pub_open_access_share_method]['id']
            res_pub_open_access_dic_share_method_code = dict_share_method[_res_pub_open_access_share_method]['code']
            res_pub_open_access_dic_share_method_pol = dict_share_method[_res_pub_open_access_share_method]['label_pl']
            res_pub_open_access_dic_share_method_eng = dict_share_method[_res_pub_open_access_share_method]['label_en']
            
            _res_pub_open_access_share_time = row['OPEN_ACCESS_RELEASE_DATE_MODE_METADATA']
            res_pub_open_access_dic_share_time_id = dict_share_time[_res_pub_open_access_share_time]['id']
            res_pub_open_access_dic_share_time_code = dict_share_time[_res_pub_open_access_share_time]['code']
            res_pub_open_access_dic_share_time_pol = dict_share_time[_res_pub_open_access_share_time]['label_pl']
            res_pub_open_access_dic_share_time_eng = dict_share_time[_res_pub_open_access_share_time]['label_en']
            
            res_pub_open_access_publicaiton_date_oa = row['OPEN_ACCESS_RELEASE_DATE_METADATA']
            res_pub_open_access_short_date_month = row['OPEN_ACCESS_RELEASE_DATE_MONTH_METADATA']
            res_pub_open_access_short_date_year = row['OPEN_ACCESS_RELEASE_DATE_YEAR_METADATA']
            res_pub_open_access_short_date = None
            if res_pub_open_access_short_date_month or res_pub_open_access_short_date_year is not None:
                res_pub_open_access_short_date = bool(res_pub_open_access_short_date_month) or bool(res_pub_open_access_short_date_year)
            
            res_pub_open_access_year = row['OPEN_ACCESS_RELEASE_DATE_YEAR_METADATA']
            _res_pub_open_access_month = row['OPEN_ACCESS_RELEASE_DATE_MONTH_METADATA']
            res_pub_open_access_month_id = dict_month[_res_pub_open_access_month]['id']
            res_pub_open_access_month_code = dict_month[_res_pub_open_access_month]['code']
            res_pub_open_access_month_label_pl = dict_month[_res_pub_open_access_month]['label_pl']
            res_pub_open_access_month_label_en = dict_month[_res_pub_open_access_month]['label_en']
            
            res_pub_open_access_month_amount = row['OPEN_ACCESS_MONTHS_METADATA']
        
            # PUBLICATION IDENTIFIERS
            res_pub_identif = []
        
            for publ_identif in group:
                identifier = publ_identif
                
                _source = 'PBN'
                dic_golden_record_source_id = dict_golden_record_source[_source]['id']
                dic_golden_record_source_code = dict_golden_record_source[_source]['code']
                dic_golden_record_source_value_pol = dict_golden_record_source[_source]['label_pl']
                dic_golden_record_source_value_eng = dict_golden_record_source[_source]['label_en']
                
                publ_identif_json = {
                    "identifier": identifier,
                    "dicGoldenRecordSource": {
                        "dicGoldenRecordSourceId": dic_golden_record_source_id,
                        "dicGoldenRecordSourceCode": dic_golden_record_source_code,
                        "dicGoldenRecordSourceLabelPl": dic_golden_record_source_value_pol,
                        "dicGoldenRecordSourceLabelEn": dic_golden_record_source_value_eng
                    }
                }
                
                res_pub_identif.append(publ_identif_json)
    
            res_pub_identif=list(unique_everseen(res_pub_identif))
        
            # PARENT PUBLICATION
            row = df_filtered_pd[df_filtered_pd['OBJECT_ID_PUBLICATION'] == best_id].iloc[0]

            book_year = row['YEAR_BOOK']
            book_doi = row['DOI_BOOK']
            book_ismn = row['ISMN_BOOK']
            book_translation = row['TRANSLATION_BOOK']
            if book_translation is not None:
                book_translation = bool(book_translation)
            book_url = row['URL_BOOK']
            book_volume = row['VOLUME_BOOK']
            book_issn = row['ISSN_BOOK']
            book_title = row['TITLE_BOOK']
            book_main_title = None
            if book_title is not None:
                book_main_title = True
            
            book_publisher = row['PUBLISHER_BOOK']
            book_publisher_status = 'ACTIVE'
            book_isbn = row['ISBN_BOOK']

            book_publishers_data = []
    
            publishers_json = {
                "publisher": {
                    "name": book_publisher,
                    "points": None,
                    "status": book_publisher_status
                },
                "resPubPublisher": {
                    "isbn": book_isbn,
                    "publishingFormat": {
                        "publishingFormatId": None,
                        "publishingFormatCode": None,
                        "publishingFormatLabelPl": None,
                        "publishingFormatLabelEn": None
                    },
                    "reedition": None,
                    "publishingLeadership": None
                }
            }
    
            book_publishers_data.append(publishers_json)
        
            book_short_date = None
            
            _book_lang_name = None
            book_cf_lang_name_id = None
            book_cf_lang_name_code = None
            book_cf_lang_name_value_pol = None
            book_cf_lang_name_value_eng = None
            book_cf_lang_name_value_ger = None
            book_cf_lang_name_value_fre = None
            book_cf_lang_name_iso_639_2 = None
            book_cf_lang_name_iso_639_1 = None
            book_cf_lang_name_wikiurl = None
            
            _book_title_lang_name = None
            book_title_cf_lang_name_id = None
            book_title_cf_lang_name_code = None
            book_title_cf_lang_name_value_pol = None
            book_title_cf_lang_name_value_eng = None
            book_title_cf_lang_name_value_ger = None
            book_title_cf_lang_name_value_fre = None
            book_title_cf_lang_name_iso_639_2 = None
            book_title_cf_lang_name_iso_639_1 = None
            book_title_cf_lang_name_wikiurl = None
            
            _book_open_access_license = row['OPEN_ACCESS_LICENSE_BOOK']
            book_dic_license_id = dict_license[_book_open_access_license]['id']
            book_dic_license_code = dict_license[_book_open_access_license]['code']
            book_dic_license_value_pol = dict_license[_book_open_access_license]['label_pl']
            book_dic_license_value_eng = dict_license[_book_open_access_license]['label_en']
            
            _book_open_access_text_version = row['OPEN_ACCESS_TEXT_VERSION_BOOK']
            book_dic_text_version_id = dict_text_version[_book_open_access_text_version]['id']
            book_dic_text_version_code = dict_text_version[_book_open_access_text_version]['code']
            book_dic_text_version_pol = dict_text_version[_book_open_access_text_version]['label_pl']
            book_dic_text_version_eng = dict_text_version[_book_open_access_text_version]['label_en']
            
            _book_open_access_share_method = row['OPEN_ACCESS_MODE_BOOK']
            book_dic_share_method_id = dict_share_method[_book_open_access_share_method]['id']
            book_dic_share_method_code = dict_share_method[_book_open_access_share_method]['code']
            book_dic_share_method_pol = dict_share_method[_book_open_access_share_method]['label_pl']
            book_dic_share_method_eng = dict_share_method[_book_open_access_share_method]['label_en']
            
            _book_open_access_share_time = row['OPEN_ACCESS_RELEASE_DATE_MODE_BOOK']
            book_dic_share_time_id = dict_share_time[_book_open_access_share_time]['id']
            book_dic_share_time_code = dict_share_time[_book_open_access_share_time]['code']
            book_dic_share_time_pol = dict_share_time[_book_open_access_share_time]['label_pl']
            book_dic_share_time_eng = dict_share_time[_book_open_access_share_time]['label_en']
            
            book_publication_date_oa = row['OPEN_ACCESS_RELEASE_DATE_BOOK']
            book_oa_year = row['OPEN_ACCESS_RELEASE_DATE_YEAR_BOOK']
            
            _book_month = row['OPEN_ACCESS_RELEASE_DATE_MONTH_BOOK']
            book_month_id = dict_month[_book_month]['id']
            book_month_code = dict_month[_book_month]['code']
            book_month_label_pl = dict_month[_book_month]['label_pl']
            book_month_label_en = dict_month[_book_month]['label_en']
            book_month_amount = row['OPEN_ACCESS_MONTHS_BOOK']
            
            _source = 'PBN'
            book_dic_golden_record_source_id = dict_golden_record_source[_source]['id']
            book_dic_golden_record_source_code = dict_golden_record_source[_source]['code']
            book_dic_golden_record_source_value_pol = dict_golden_record_source[_source]['label_pl']
            book_dic_golden_record_source_value_eng = dict_golden_record_source[_source]['label_en']
            
            book_identifier = None
            book_deduplication_publication_group_identifier = None
            if book_title is not None:
                book_deduplication_publication_group_identifier = str(uuid.uuid4())
    
            book_source_type = general_source_type
            book_dic_source_type_id = dict_source_type[book_source_type]['id']
            book_dic_source_type_code = dict_source_type[book_source_type]['code']
            book_dic_source_type_value_pol = dict_source_type[book_source_type]['label_pl']
            book_dic_source_type_value_eng = dict_source_type[book_source_type]['label_en']
    
            book_publication_type = "BOOK"
            book_publication_type_id = dict_publication_type[book_publication_type]['id']
            book_publication_type_code = dict_publication_type[book_publication_type]['code']
            book_publication_value_pol = dict_publication_type[book_publication_type]['label_pl']
            book_publication_value_eng = dict_publication_type[book_publication_type]['label_en']
            
            publ_identif_json = {
                "identifier": book_identifier,
                "dicGoldenRecordSource": {
                    "dicGoldenRecordSourceId": book_dic_golden_record_source_id,
                    "dicGoldenRecordSourceCode": book_dic_golden_record_source_code,
                    "dicGoldenRecordSourceLabelPl": book_dic_golden_record_source_value_pol,
                    "dicGoldenRecordSourceLabelEn": book_dic_golden_record_source_value_eng
                }
            }
            
            book_res_pub_identif = [
                publ_identif_json
            ]
    
            book_res_pub_identif=list(unique_everseen(book_res_pub_identif))

            # JSON
            _json = {
                "actionType": action_type,
                "authorProfileIdn": author_profile_idn,
                "creationTimestamp": date_time_string,
                "jsonVersion": json_version,
                "globalPublicationId": global_id,
                "publicationId": deduplication_publication_group_identifier,
                "parentPublicationId": book_deduplication_publication_group_identifier,
                "mainPublication": {
                    "personsData": persons_data,
                    "cfResPubl": {
                        "pageNumer": page_number,
                        "year": year,
                        "pageFrom": page_from,
                        "pageTo": page_to,
                        "doi": doi,
                        "ismn": ismn,
                        "translation": translation,
                        "url": url,
                        "volumeNumber": volume_number,
                        "issueNumber": issue_number,
                        "publishingSeries": publishing_series,
                        "publishingSeriesNumber": publishing_series_number,
                        "publishingNumber": publishing_number,
                        "publicationPlace": publication_place,
                        "cfLangName": {
                            "cfLangNameId": publication_cf_lang_name_id,
                            "cfLangNameCode": publication_cf_lang_name_code,
                            "cfLangNameLanguagePl": publication_cf_lang_name_value_pol,
                            "cfLangNameLanguageEn": publication_cf_lang_name_value_eng,
                            "cfLangNameLanguageDe": publication_cf_lang_name_value_ger,
                            "cfLangNameLanguageFr": publication_cf_lang_name_value_fre,
                            "cfLangNameIso639_2": publication_cf_lang_name_iso_639_2,
                            "cfLangNameIso639_1": publication_cf_lang_name_iso_639_1,
                            "cfLangNameWikiurl": publication_cf_lang_name_wikiurl
                        },
                        "dicPublicationType": {
                            "dicPublicationTypeId": dic_publication_type_id,
                            "dicPublicationTypeCode": dic_publication_type_code,
                            "dicPublicationTypeLabelPl": dic_publication_type_value_pol,
                            "dicPublicationTypeLabelEn": dic_publication_type_value_eng
                        },
                        "issn": issn,
                        "dicSourceType": {
                            "dicSourceTypeId": dic_source_type_id,
                            "dicSourceTypeCode": dic_source_type_code,
                            "dicSourceTypeLabelPl": dic_source_type_value_pol,
                            "dicSourceTypeLabelEn": dic_source_type_value_eng
                        },
                    },
                    "cfResPublAbstr": cf_res_publ_abstract,
                    "cfResPublBiblNote": cf_res_publ_bibl_note,
                    "cfResPublKeywords": cf_res_publ_keyw,
                    "cfResTitle": cf_res_title,
                    "journals": {
                        "name": journal_name,
                        "issn": journal_issn,
                        "eissn": journal_eissn,
                        "points": journalIsPoints,
                        "url": journal_url,
                        "publisher": {
                            "name": journal_publisher_name,
                            "points": journal_publisher_points,
                            "status": journal_publisher_status
                        }
                    },
                    "conferenceSeries": {
                        "name": conference_series_name,
                        "points": conferenceSeriesIsPoints,
                        "shortName": conference_series_short_name,
                        "url": conference_series_url
                    },
                    "conferenceEvent": {
                        "fullName": conference_event_full_name,
                        "startDate": conference_event_start_date,
                        "place": conference_event_place,
                        "endDate": conference_event_end_date,
                        "url": conference_event_url,
                        "shortName": conference_event_short_name,
                        "country": {
                            "countryId": conference_event_country_id,
                            "countryCode": conference_event_country_code,
                            "countryLabelPl": conference_event_country_value_pol,
                            "countryLabelEn": conference_event_country_value_eng
                        },
                        "conferenceEventType": {
                            "conferenceEventTypeId": conference_event_conference_event_type_id,
                            "conferenceEventTypeCode": conference_event_conference_event_type_code,
                            "conferenceEventTypeLabelPl": conference_event_conference_event_type_value_pol,
                            "conferenceEventTypeLabelEn": conference_event_conference_event_type_value_eng
                        }
                    },
                    "publishersData": publishers_data,
                    "publishingPlatform": {
                        "publishingPlatformName": publishing_platform_name
                    },
                    "points": {
                        "points": points_points,
                        "year": points_year
                    },
                    "resPubOpenAccess": {
                        "dicLicense": {
                            "dicLicenseId": res_pub_open_access_dic_license_id,
                            "dicLicenseCode": res_pub_open_access_dic_license_code,
                            "dicLicenseLabelPl": res_pub_open_access_dic_license_pol,
                            "dicLicenseLabelEn": res_pub_open_access_dic_license_eng
                        },
                        "dicTextVersion": {
                            "dicTextVersionId": res_pub_open_access_dic_text_version_id,
                            "dicTextVersionCode": res_pub_open_access_dic_text_version_code,
                            "dicTextVersionLabelPl": res_pub_open_access_dic_text_version_pol,
                            "dicTextVersionLabelEn": res_pub_open_access_dic_text_version_eng
                        },
                        "dicShareMethod": {
                            "dicShareMethodId": res_pub_open_access_dic_share_method_id,
                            "dicShareMethodCode": res_pub_open_access_dic_share_method_code,
                            "dicShareMethodLabelPl": res_pub_open_access_dic_share_method_pol,
                            "dicShareMethodLabelEn": res_pub_open_access_dic_share_method_eng
                        },
                        "dicShareTime": {
                            "dicShareTimeId": res_pub_open_access_dic_share_time_id,
                            "dicShareTimeCode": res_pub_open_access_dic_share_time_code,
                            "dicShareTimeLabelPl": res_pub_open_access_dic_share_time_pol,
                            "dicShareTimeLabelEn": res_pub_open_access_dic_share_time_eng
                        },
                        "publicationDateOa": res_pub_open_access_publicaiton_date_oa,
                        "shortDate": res_pub_open_access_short_date,
                        "year": res_pub_open_access_year,
                        "dicMonth": {
                            "dicMonthId": res_pub_open_access_month_id,
                            "dicMonthCode": res_pub_open_access_month_code,
                            "dicMonthLabelPl": res_pub_open_access_month_label_pl,
                            "dicMonthLabelEn": res_pub_open_access_month_label_en,
                        },
                        "monthAmount": res_pub_open_access_month_amount
                    },
                    "resPubIdentif": res_pub_identif,
                    "publicationGroupId": deduplication_publication_group_identifier
                },
                "parentPublication": {
                    "cfResPubl": {
                        "pageNumer": None,
                        "year": book_year,
                        "pageFrom": None,
                        "pageTo": None,
                        "doi": book_doi,
                        "ismn": book_ismn,
                        "translation": book_translation,
                        "url": book_url,
                        "volumeNumber": book_volume,
                        "issueNumber": None,
                        "publishingSeries": None,
                        "publishingSeriesNumber": None,
                        "publishingNumber": None,
                        "publicationPlace": None,
                        "cfLangName": {
                            "cfLangNameId": book_cf_lang_name_id,
                            "cfLangNameCode": book_cf_lang_name_code,
                            "cfLangNameLanguagePl": book_cf_lang_name_value_pol,
                            "cfLangNameLanguageEn": book_cf_lang_name_value_eng,
                            "cfLangNameLanguageDe": book_cf_lang_name_value_ger,
                            "cfLangNameLanguageFr": book_cf_lang_name_value_fre,
                            "cfLangNameIso639_2": book_cf_lang_name_iso_639_2,
                            "cfLangNameIso639_1": book_cf_lang_name_iso_639_1,
                            "cfLangNameWikiurl": book_cf_lang_name_wikiurl
                        },
                        "dicPublicationType": {
                            "dicPublicationTypeId": book_publication_type_id,
                            "dicPublicationTypeCode": book_publication_type_code,
                            "dicPublicationTypeLabelPl": book_publication_value_pol,
                            "dicPublicationTypeLabelEn": book_publication_value_eng
                        },
                        "issn": book_issn,
                        "dicSourceType": {
                            "dicSourceTypeId": book_dic_source_type_id,
                            "dicSourceTypeCode": book_dic_source_type_code,
                            "dicSourceTypeLabelPl": book_dic_source_type_value_pol,
                            "dicSourceTypeLabelEn": book_dic_source_type_value_eng
                        }
                        
                    },
                    "cfResTitle": [
                        {
                            "title": book_title,
                            "mainTitle": book_main_title,
                            "cfLangName": {
                                "cfLangNameId": book_title_cf_lang_name_id,
                                "cfLangNameCode": book_title_cf_lang_name_code,
                                "cfLangNameLanguagePl": book_title_cf_lang_name_value_pol,
                                "cfLangNameLanguageEn": book_title_cf_lang_name_value_eng,
                                "cfLangNameLanguageDe": book_title_cf_lang_name_value_ger,
                                "cfLangNameLanguageFr": book_title_cf_lang_name_value_fre,
                                "cfLangNameIso639_2": book_title_cf_lang_name_iso_639_2,
                                "cfLangNameIso639_1": book_title_cf_lang_name_iso_639_1,
                                "cfLangNameWikiurl": book_title_cf_lang_name_wikiurl
                            },
                            "dicSourceType": {
                                "dicSourceTypeId": book_dic_source_type_id,
                                "dicSourceTypeCode": book_dic_source_type_code,
                                "dicSourceTypeLabelPl": book_dic_source_type_value_pol,
                                "dicSourceTypeLabelEn": book_dic_source_type_value_eng
                            },
                            "translation": None
                        }
                    ],
                    "publishersData": book_publishers_data,
                    "resPubOpenAccess": {
                        "dicLicense": {
                            "dicLicenseId": book_dic_license_id,
                            "dicLicenseCode": book_dic_license_code,
                            "dicLicenseLabelPl": book_dic_license_value_pol,
                            "dicLicenseLabelEn": book_dic_license_value_eng
                        },
                        "dicTextVersion": {
                            "dicTextVersionId": book_dic_text_version_id,
                            "dicTextVersionCode": book_dic_text_version_code,
                            "dicTextVersionLabelPl": book_dic_text_version_pol,
                            "dicTextVersionLabelEn": book_dic_text_version_eng
                        },
                        "dicShareMethod": {
                            "dicShareMethodId": book_dic_share_method_id,
                            "dicShareMethodCode": book_dic_share_method_code,
                            "dicShareMethodLabelPl": book_dic_share_method_pol,
                            "dicShareMethodLabelEn": book_dic_share_method_eng
                        },
                        "dicShareTime": {
                            "dicShareTimeId": book_dic_share_time_id,
                            "dicShareTimeCode": book_dic_share_time_id,
                            "dicShareTimeLabelPl": book_dic_share_time_pol,
                            "dicShareTimeLabelEn": book_dic_share_time_eng
                        },
                        "publicationDateOa": book_publication_date_oa,
                        "shortDate": book_short_date,
                        "year": book_oa_year,
                        "dicMonth": {
                            "dicMonthId": book_month_id,
                            "dicMonthCode": book_month_code,
                            "dicMonthLabelPl": book_month_label_pl,
                            "dicMonthLabelEn": book_month_label_en,
                        },
                        "monthAmount": book_month_amount
                    },
                "resPubIdentif": book_res_pub_identif,
                "publicationGroupId": book_deduplication_publication_group_identifier
                }
            }
    
        except Exception as e:
            print("Traceback: ", traceback.format_exc())
            excep = str(traceback.format_exc())

        json_string = ""
        
        if isinstance(_json, dict):
            _json = remove_nested_none_objects(_json)
            json_string = stringify_json(_json)
        
        return pdf.assign(JSON = json_string, ERROR = excep, PUBLICATION_ID = uuid_name, GLOBAL_PUBLICATION_ID = global_id)

    return df.groupby("NEW_UUID").apply(gen_json)

In [48]:
# Read lookup data
t0 = time.time()
data_lookup = spark.read.format("delta").load("hdfs://hadoop:9000/hd_micula/delta_jsons/data_lookup")
t1 = time.time()
total = t1-t0
print(total)

7.449153900146484


In [56]:
t0 = time.time()
generated_data = gen_json_wrap(data_lookup, data=data, action_type="INSERT")
# generated_data.cache()
# generated_data.persist()
t1 = time.time()
total = t1-t0
print('LOG: PROPER TASK SETUP TIME', total)

/home/hd_micula/.local/lib/python3.9/site-packages/pyspark/sql/pandas/group_ops.py:103: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


LOG: PROPER TASK SETUP TIME 46.67968463897705


In [ ]:
# Save to parquet
t0 = time.time()
# generated_data.write.format("delta").save(f"hdfs://hadoop:9000/hd_micula/delta_jsons/test_v1")
generated_data.write.format("parquet").mode("overwrite").save(f"hdfs://hadoop:9000/hd_micula/delta_jsons/test_v1")
t1 = time.time()
total = t1-t0
print('LOG: SAVE TASK RESULTS TIME', total)

In [65]:
# Read and check Data
t0 = time.time()
read_data = spark.read.format("delta").load("hdfs://hadoop:9000/hd_micula/delta_jsons/generated_jsons_v77_25_000")
t1 = time.time()
total = t1-t0
print(total)

0.020937442779541016


In [66]:
t0 = time.time()
print(read_data.count())
t1 = time.time()
total = t1-t0
print(total)

25000
0.2666306495666504


In [68]:
t0 = time.time()
read_data.show(3)
t1 = time.time()
total = t1-t0
print(total)

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+---------------------+
|                UUID|           OBJECT_ID|          PERSON_HSK|            NEW_UUID|                JSON|ERROR|      PUBLICATION_ID|GLOBAL_PUBLICATION_ID|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+---------------------+
|48060D5EFE39DC5CE...|5ec00197ad49b31cc...|BEA86E8338C8FDAC6...|0000c5a5-782c-455...|{"in_json": {"act...|     |0000c5a5-782c-455...| 0000c5a5-782c-455...|
|63C8DCF275F8E177B...|5f68b5fd2467f04a3...|A58DBBE7B243B78C1...|00022201-8e4f-4b6...|{"in_json": {"act...|     |00022201-8e4f-4b6...| 00022201-8e4f-4b6...|
|70D73D6835395698A...|615558f32467f025d...|C0B77039F2B8E9B9D...|00047fcd-2fc0-441...|{"in_json": {"act...|     |00047fcd-2fc0-441...| 00047fcd-2fc0-441...|
+--------------------+--------------------+--------------------+

In [ ]:
data_lookup.repartition(5)
# generated_data = gen_json_wrap(temp_df, data=data, action_type="INSERT")
data_lookup.show(10)
